In [1]:
import os 
os.chdir(r'D:\pendrive\Projects links\CTC_prediction')

In [2]:
import numpy as np
import pandas as pd

In [3]:
mtr=pd.read_csv("train.csv",header=0)
mte=pd.read_csv("test.csv",header=0)

In [4]:
print(mtr.shape)
mtr.head()
mte.shape

(26415, 34)


(10875, 33)

In [5]:
print(mtr.duplicated().sum())
print(mtr.isnull().sum().sum())

0
0


In [6]:

mtr=mtr.replace(-1,np.nan)
mtr=mtr.replace(0,np.nan)
mtr=mtr.replace(-100,0)

mte=mte.replace(-1,np.nan)
mte=mte.replace(0,np.nan)
mte=mte.replace(-100,0)


In [7]:
print(mtr.isnull().sum())

Candidate ID                          0
Pay_in_INR                            0
Gender                                0
Date Of Birth                         0
Score in Tenth                        0
School Board in Tenth                 0
Year Of Twelth Completion             0
Score in Twelth                       0
Board in Twelth                       0
CollegeCode                           0
CollegeTier                           0
Graduation                            0
Discipline                            0
GPA Score in Graduation               0
CityCode                              0
CityTier                          18314
State                                 0
Year of Graduation Completion         0
Score in English language             0
Score in Logical skill                0
Score in Quantitative ability         0
Score in Domain                     732
Score in ComputerProgramming          0
Score in ElectronicsAndSemicon        0
Score in ComputerScience              0


In [8]:
mtr=mtr.fillna(mtr.mean())
mte=mte.fillna(mte.mean())
print(mte.isna().sum().sum())

0


In [9]:
mtr=mtr.drop(columns=['CityTier','Candidate ID','Date Of Birth'])
mte=mte.drop(columns=['CityTier','Candidate ID','Date Of Birth'])



In [10]:
mtr.dtypes

Pay_in_INR                        float64
Gender                             object
Score in Tenth                    float64
School Board in Tenth              object
Year Of Twelth Completion           int64
Score in Twelth                   float64
Board in Twelth                    object
CollegeCode                         int64
CollegeTier                         int64
Graduation                         object
Discipline                         object
GPA Score in Graduation           float64
CityCode                            int64
State                              object
Year of Graduation Completion       int64
Score in English language           int64
Score in Logical skill              int64
Score in Quantitative ability       int64
Score in Domain                   float64
Score in ComputerProgramming        int64
Score in ElectronicsAndSemicon      int64
Score in ComputerScience            int64
Score in MechanicalEngg             int64
Score in ElectricalEngg           

In [11]:
x=mtr.drop(columns=['Pay_in_INR'])
y=mtr['Pay_in_INR']

In [12]:
cat=[col for col in x.columns if x[col].dtype == 'object']
print(cat)

testw=[colu for colu in mte.columns if mte[colu].dtype=='object']
testw

['Gender', 'School Board in Tenth', 'Board in Twelth', 'Graduation', 'Discipline', 'State']


['Gender',
 'School Board in Tenth',
 'Board in Twelth',
 'Graduation',
 'Discipline',
 'State']

In [13]:
x=pd.get_dummies(x,columns =cat)
test=pd.get_dummies(mte,columns=testw)

from sklearn.datasets import load_digits
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import chi2,SelectKBest

y=mtr['Pay_in_INR']
x=mtr.drop(columns='Pay_in_INR')

sigfea=SelectKBest(score_func=f_regression,k=10)
fits = sigfea.fit(x,y)
chi_support=sigfea.get_support()
chi_features = x.loc[:,chi_support].columns.tolist()
print(chi_features)

In [14]:
from sklearn import preprocessing as pps
std=pps.StandardScaler().fit(x[:]).transform(x[:])
x.head()

from sklearn import preprocessing as pps
std=pps.StandardScaler().fit(test[:]).transform(test[:])
test.head() 

,Score in Tenth,Year Of Twelth Completion,Score in Twelth,CollegeCode,CollegeTier,GPA Score in Graduation,CityCode,Year of Graduation Completion,Score in English language,Score in Logical skill,...,State_Meghalaya,State_Orissa,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal
0,78.00,2008,76.80,11368,2,73.90,11368,2012,545,625,...,0,0,0,0,0,0,0,1,0,0
1,89.92,2010,87.00,5086,2,76.32,5086,2014,560,555,...,0,0,0,0,0,0,0,0,0,0
2,60.80,2006,66.66,41,2,74.74,41,2013,465,585,...,0,0,0,0,0,0,0,0,0,0
3,88.00,2010,90.00,9173,2,76.00,9173,2014,625,555,...,0,0,0,0,0,0,1,0,0,0
4,86.80,2008,82.80,9019,2,78.00,9019,2012,575,585,...,0,0,0,0,0,0,0,0,0,0


In [15]:
print(x.shape)
print(test.shape)

(26415, 694)
(10875, 290)


In [16]:
r=[c for c in x.columns  if c in test.columns]
f=x[r]
x=pd.DataFrame(f)

In [17]:
x.shape

(26415, 290)

In [18]:
test.shape

(10875, 290)

# Splitting and Model building 

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=20)       

In [20]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
print(test.shape)
print(type(x_train),type(x_test),type(y_train),type(y_test))


(19811, 290) (19811,) (6604, 290) (6604,)
(10875, 290)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [21]:
from sklearn import tree 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
deci=tree.DecisionTreeRegressor(criterion='mse') # ,max_depth=3
deci_fit=deci.fit(x_train,y_train)
params = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          'max_depth': [i for i in range(1,10)], 
          #'n_estimators' : [50.100,150,200,250,400,500],
          'random_state':[123]}
rf_random = RandomizedSearchCV(estimator = deci, param_distributions = params, n_iter = 100, cv = 3, verbose=0, random_state=42)
rf_random.fit(x_train,y_train)
#print('Best Penalty:', rf_random.best_estimator_.get_params())
rf_random.best_params_ 


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error 
from sklearn.tree import DecisionTreeRegressor

deci1=tree.DecisionTreeRegressor(criterion='mse',max_depth=9,random_state=123, 
                                  min_samples_split=2, min_samples_leaf= 7, max_features= 'auto')
deci_fit=deci1.fit(x_train,y_train)

deci_31 = deci1.predict(test)

pre_test=deci1.predict(x_test)
deci_ =mean_squared_error(y_test,pre_test) 
deci_ 

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

clf = xgb.XGBRegressor()
param_grid = {
        
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [50]}

xgboo = RandomizedSearchCV(clf, param_grid, n_iter=10,
                            n_jobs=2, verbose=1, cv=4, random_state=42)
xgboo.fit(x_train,y_train)
xgboo.best_params_ 

In [ ]:
clf

In [ ]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb
xgbo=xgb.XGBRegressor(n_estimators=100,subsample= 0.6,
 reg_lambda= 1.0,
 
 min_child_weight=1.0,
 max_depth=20,
 learning_rate= 0.1,
 gamma= 0.25,
 colsample_bytree= 0.8,
 colsample_bylevel= 0.8 )
xgbo.fit(x_train,y_train)

xg_pre=xgbo.predict(test)
x_pre=xgbo.predict(x_test)
xgb_mse='xgboost acc',mean_squared_error(y_test,x_pre)
xgb_mse 

In [ ]:
from sklearn import svm 
from sklearn.model_selection import RandomizedSearchCV
model=svm.SVC()
para={'kernel':['rbf','linear'],'C':[0.001,0.1,0.01,1.0,10.0,100.0,2.0], 'gamma':([i for i in range(1,3)],0.1)}
ra_svm=RandomizedSearchCV(model,para,cv=3,random_state=15,verbose=1)
ra_svm.fit(x_train,y_train)
ra_svm.best_params_  

In [ ]:
from sklearn import svm
svc = svm.SVC(kernel='linear', C=1,gamma=1).fit(x_train, y_train)
svc_pre=svc.predict(test)
val_pre=svc.predict(x_test)
svm_score=mean_sqauared_error(y_test,val_pre)
svm_score

In [52]:
from sklearn.ensemble import RandomForestRegressor

model3=RandomForestRegressor() 

para_rf= {'criterion':'mse', 'max_depth':([i for i in range(1,12)]),
                      'max_features':["auto", "sqrt","log2"], 'max_leaf_nodes':([j for j in range(1,12)]),
                      
                      'min_samples_leaf':([k for k in range(1,12)]), 'min_samples_split':([l for l in range(1,12)])}
ran_rf=RandomizedSearchCV(model3,para_rf,random_state=14,n_jobs=-1,cv=4,verbose=2)


In [53]:

ran_rf.fit(x_train,y_train)
ran_rf.best_params_

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyError: 'e'